<a href="https://colab.research.google.com/github/Akasxh/Mail-Merge/blob/main/Mail_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai

In [ ]:
# Run these in a Colab notebook
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas

In [ ]:
pip install --upgrade gspread google-auth oauth2client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.36.0 which is incompatible.


In [ ]:
from google.colab import files

# Upload credentials.json
uploaded = files.upload()

# Verify file upload
for filename, content in uploaded.items():
    print(f'Uploaded {filename} ({len(content)} bytes)')

Saving credentials.json to credentials (1).json
Uploaded credentials (1).json (414 bytes)


In [ ]:
import pandas as pd
import base64
from google.colab import auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from email.mime.text import MIMEText
import os

In [ ]:
import os
import google.generativeai as genai
import pandas as pd
from google.colab import auth
from googleapiclient.discovery import build

gsheet = None  # Global variable to store the selected Google Sheets information (name and ID)

def authenticate_and_get_services():
    """Authenticate and return Drive and Sheets services"""
    auth.authenticate_user()
    from google.auth import default
    credentials, _ = default(
        scopes=[
            'https://www.googleapis.com/auth/drive',
            'https://www.googleapis.com/auth/spreadsheets'
        ]
    )

    drive_service = build('drive', 'v3', credentials=credentials)
    sheets_service = build('sheets', 'v4', credentials=credentials)

    return drive_service, sheets_service

def list_available_spreadsheets(drive_service):
    """List available Google Sheets"""
    results = drive_service.files().list(
        q="mimeType='application/vnd.google-apps.spreadsheet'",
        spaces='drive',
        fields="files(id, name)"
    ).execute()

    spreadsheets = results.get('files', [])

    print("Available Spreadsheets:")
    for i, sheet in enumerate(spreadsheets):
        print(f"{i+1}. {sheet['name']} (ID: {sheet['id']})")

    return spreadsheets

def fetch_sheet_data(sheets_service, spreadsheet_id):
    """Fetch data from selected spreadsheet"""
    try:
        result = sheets_service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range='Sheet1'
        ).execute()

        data = result.get('values', [])

        if not data:
            raise ValueError("No data found in sheet")

        headers = data[0]
        df = pd.DataFrame(data[1:], columns=headers)

        return df

    except Exception as e:
        print(f"Error fetching sheet data: {e}")
        return None

def generate_gemini_response(topic, api_key):
    """Generate response using Gemini API"""
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel('gemini-pro')

    prompt = f"Write a engaging message about the topic: {topic}. Make it awesome."

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating Gemini response: {e}")
        return topic

def get_sheet_url(sheet_id):
    """Generate the URL for the Google Sheet using its ID"""
    return f"https://docs.google.com/spreadsheets/d/{sheet_id}/edit"

def update_sheet_with_gemini_response(sheets_service, spreadsheet_id, df, gemini_responses):
    """Update sheet with Gemini generated responses"""
    # Prepare updated data
    updated_data = df.copy()
    updated_data['gemini_response'] = gemini_responses

    # Convert DataFrame back to list of lists
    updated_values = [df.columns.tolist()] + updated_data.values.tolist()

    # Update sheet
    body = {
        'values': updated_values
    }

    try:
        result = sheets_service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range='Sheet1',
            valueInputOption='RAW',
            body=body
        ).execute()
        print("Sheet updated successfully!")
    except Exception as e:
        print(f"Error updating sheet: {e}")

def main():
    # Get Gemini API Key
    gemini_api_key = input("Enter your Gemini API key: ")

    # Authenticate and get services
    drive_service, sheets_service = authenticate_and_get_services()

    # List and select spreadsheet
    spreadsheets = list_available_spreadsheets(drive_service)
    selection = int(input("Enter the number of the spreadsheet you want to use: ")) - 1
    selected_sheet = spreadsheets[selection]

    # Store the selected sheet globally
    global gsheet
    gsheet = selected_sheet
    print(f"Selected Google Sheet: {gsheet['name']} (ID: {gsheet['id']})")
    print(f"Sheet URL: {get_sheet_url(gsheet['id'])}")

    # Fetch sheet data
    sheet_data = fetch_sheet_data(sheets_service, gsheet['id'])

    if sheet_data is not None:
        # Generate Gemini responses for topics
        gemini_responses = sheet_data['topic'].apply(
            lambda topic: generate_gemini_response(topic, gemini_api_key)
        )

        # Update sheet with Gemini responses
        update_sheet_with_gemini_response(
            sheets_service,
            gsheet['id'],
            sheet_data,
            gemini_responses
        )

# Run the main function
if __name__ == '__main__':
    main()


In [6]:
from google.colab import files
uploaded = files.upload()  # This lets you manually upload your token.pickle file


Saving token.pickle to token.pickle


In [7]:
import pickle
import os
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Load the token
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

# Refresh the token if it's expired
if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())

# Now you can use the creds object to build your Google API services
service = build('gmail', 'v1', credentials=creds)


In [8]:
# Authenticate with Google Sheets using token.pickle
gc = gspread.authorize(creds)

In [9]:
# Open the Google Sheets document by ID
sheet = gc.open_by_key(gsheet['id']).sheet1  # Use the sheet ID from gsheet variable

In [10]:
# Fetch email data from Google Sheets columns
name_column = sheet.col_values(1)  # Column A: Name
email_column = sheet.col_values(2)  # Column B: Email
topics_column = sheet.col_values(3)  # Column C: Topics
data_column = sheet.col_values(4)  # Column D: Data

In [12]:
import time
# Function to send an email using Gmail API
def send_email(to_email, subject, body_text):
    message = MIMEText(body_text)
    message['to'] = to_email
    message['subject'] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    message = {'raw': raw}

    try:
        sent_message = service.users().messages().send(userId='me', body=message).execute()
        print(f"Email sent to {to_email}. Message Id: {sent_message['id']}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Throttling delay (in seconds) between each email
SEND_DELAY = 5  # Delay between emails (adjust as needed)

# Loop through the emails and topics to send messages
for name, email, topic, data in zip(name_column, email_column, topics_column, data_column):
    email_body = f"""
    Hello {name},

    {data}

    Best regards,
    """

    # Send email
    send_email(
        to_email=email,
        subject=f"About {topic}",
        body_text=email_body
    )

    # Wait before sending the next email (to avoid throttling issues)
    print(f"Waiting {SEND_DELAY} seconds before sending the next email...")
    time.sleep(SEND_DELAY)

An error occurred: <HttpError 400 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/send?alt=json returned "Invalid To header". Details: "[{'message': 'Invalid To header', 'domain': 'global', 'reason': 'invalidArgument'}]">
Waiting 5 seconds before sending the next email...
Email sent to drakathakash@gmail.com. Message Id: 19342e81d4a60e41
Waiting 5 seconds before sending the next email...
Email sent to drakathabishek@gmail.com. Message Id: 19342e8331c4aba9
Waiting 5 seconds before sending the next email...
